<a href="https://colab.research.google.com/github/jamilahmedsagor/Recipe-Management-System/blob/main/recipe%20management%20system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

csv file upload

In [4]:
from google.colab import files
uploaded = files.upload()

Saving main.py to main.py
Saving recipe.py to recipe.py
Saving recipe_manager.py to recipe_manager.py


In [2]:
import pandas as pd
df = pd.read_csv('recipes.csv')
print(df)

   Recipe_ID Recipe_Name   Ingredients  Steps Cuisine  Preparation_Time  \
0          1     nodoles  egg,oil,salt      5  indian                10   

   Calories  
0      1200  


main.py

In [ ]:
from recipe_manager import RecipeManager
from recipe import Recipe
from tabulate import tabulate
import sys

class RecipeManagementSystem:
    def __init__(self):
        self.manager = RecipeManager()

    def display_menu(self):
        print("\n=== Recipe Management System ===")
        print("1. Add Recipe")
        print("2. Update Recipe")
        print("3. Delete Recipe")
        print("4. Search Recipes")
        print("5. Generate Shopping List")
        print("6. List All Recipes")
        print("7. Exit")

    def get_recipe_input(self):
        recipe_id = input("Enter Recipe ID: ")
        name = input("Enter Recipe Name: ")
        ingredients = input("Enter Ingredients (comma-separated): ").split(',')
        steps = input("Enter Steps (comma-separated): ").split(',')
        cuisine = input("Enter Cuisine: ")
        prep_time = int(input("Enter Preparation Time (minutes): "))
        calories = input("Enter Calories (optional, press enter to skip): ")
        calories = int(calories) if calories else None

        return Recipe(recipe_id, name, ingredients, steps, cuisine, prep_time, calories)

    def display_recipes(self, df):
        if df.empty:
            print("No recipes found!")
            return
        print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

    def run(self):
        while True:
            self.display_menu()
            choice = input("\nEnter your choice (1-7): ")

            if choice == '1':
                try:
                    recipe = self.get_recipe_input()
                    self.manager.add_recipe(recipe)
                    print("Recipe added successfully!")
                except Exception as e:
                    print(f"Error adding recipe: {e}")

            elif choice == '2':
                recipe_id = input("Enter Recipe ID to update: ")
                try:
                    recipe = self.get_recipe_input()
                    self.manager.update_recipe(recipe_id, recipe)
                    print("Recipe updated successfully!")
                except Exception as e:
                    print(f"Error updating recipe: {e}")

            elif choice == '3':
                recipe_id = input("Enter Recipe ID to delete: ")
                try:
                    self.manager.delete_recipe(recipe_id)
                    print("Recipe deleted successfully!")
                except Exception as e:
                    print(f"Error deleting recipe: {e}")

            elif choice == '4':
                print("\nSearch by:")
                print("1. Name")
                print("2. Cuisine")
                print("3. Ingredients")
                print("4. Preparation Time")

                search_choice = input("Enter your choice (1-4): ")

                if search_choice == '1':
                    name = input("Enter recipe name to search: ")
                    results = self.manager.search_by_name(name)
                elif search_choice == '2':
                    cuisine = input("Enter cuisine to search: ")
                    results = self.manager.search_by_cuisine(cuisine)
                elif search_choice == '3':
                    ingredient = input("Enter ingredient to search: ")
                    results = self.manager.search_by_ingredients(ingredient)
                elif search_choice == '4':
                    max_time = int(input("Enter maximum preparation time (minutes): "))
                    results = self.manager.filter_by_prep_time(max_time)
                else:
                    print("Invalid choice!")
                    continue

                self.display_recipes(results)

            elif choice == '5':
                recipe_ids = input("Enter Recipe IDs (comma-separated): ").split(',')
                available = input("Enter available ingredients (comma-separated): ").split(',')
                shopping_list = self.manager.generate_shopping_list(recipe_ids, available)
                print("\nShopping List:")
                for item in shopping_list:
                    print(f"- {item}")

            elif choice == '6':
                self.display_recipes(self.manager.df)

            elif choice == '7':
                print("Thank you for using Recipe Management System!")
                sys.exit(0)

            else:
                print("Invalid choice! Please try again.")

if __name__ == "__main__":
    app = RecipeManagementSystem()
    app.run()



=== Recipe Management System ===
1. Add Recipe
2. Update Recipe
3. Delete Recipe
4. Search Recipes
5. Generate Shopping List
6. List All Recipes
7. Exit

Enter your choice (1-7): 1
Enter Recipe ID: 2
Enter Recipe Name: peddis
Enter Ingredients (comma-separated): onion, salt, wheat
Enter Steps (comma-separated): 2
Enter Cuisine: bangladeshi
Enter Preparation Time (minutes): 10
Enter Calories (optional, press enter to skip): 100
Recipe added successfully!

=== Recipe Management System ===
1. Add Recipe
2. Update Recipe
3. Delete Recipe
4. Search Recipes
5. Generate Shopping List
6. List All Recipes
7. Exit

Enter your choice (1-7): 6
+-------------+---------------+--------------------+---------+-------------+--------------------+------------+
|   Recipe_ID | Recipe_Name   | Ingredients        |   Steps | Cuisine     |   Preparation_Time |   Calories |
+=============+===============+====================+=========+=============+====================+============+
|           1 | nodoles   

recipe.py

In [ ]:
class Recipe:
    def __init__(self, recipe_id, name, ingredients, steps, cuisine, prep_time, calories=None):
        self.recipe_id = recipe_id
        self.name = name
        self.ingredients = ingredients
        self.steps = steps
        self.cuisine = cuisine
        self.prep_time = prep_time
        self.calories = calories

    def to_dict(self):
        return {
            'Recipe_ID': self.recipe_id,
            'Recipe_Name': self.name,
            'Ingredients': ','.join(self.ingredients),
            'Steps': ','.join(self.steps),
            'Cuisine': self.cuisine,
            'Preparation_Time': self.prep_time,
            'Calories': self.calories
        }

    @classmethod
    def from_dict(cls, data):
        return cls(
            data['Recipe_ID'],
            data['Recipe_Name'],
            data['Ingredients'].split(','),
            data['Steps'].split(','),
            data['Cuisine'],
            data['Preparation_Time'],
            data.get('Calories')
        )


re.TEMPLATE


In [ ]:
import pandas as pd
from recipe import Recipe

class RecipeManager:
    def __init__(self, csv_file='recipes.csv'):
        self.csv_file = csv_file
        try:
            self.df = pd.read_csv(csv_file)
        except FileNotFoundError:
            self.df = pd.DataFrame(columns=['Recipe_ID', 'Recipe_Name', 'Ingredients',
                                          'Steps', 'Cuisine', 'Preparation_Time', 'Calories'])
            self.df.to_csv(csv_file, index=False)

    def add_recipe(self, recipe):
        recipe_dict = recipe.to_dict()
        self.df = pd.concat([self.df, pd.DataFrame([recipe_dict])], ignore_index=True)
        self.save()

    def update_recipe(self, recipe_id, recipe):
        self.df.loc[self.df['Recipe_ID'] == recipe_id] = pd.Series(recipe.to_dict())
        self.save()

    def delete_recipe(self, recipe_id):
        self.df = self.df[self.df['Recipe_ID'] != recipe_id]
        self.save()

    def search_by_name(self, name):
        return self.df[self.df['Recipe_Name'].str.contains(name, case=False, na=False)]

    def search_by_cuisine(self, cuisine):
        return self.df[self.df['Cuisine'].str.contains(cuisine, case=False, na=False)]

    def search_by_ingredients(self, ingredient):
        return self.df[self.df['Ingredients'].str.contains(ingredient, case=False, na=False)]

    def filter_by_prep_time(self, max_time):
        return self.df[self.df['Preparation_Time'] <= max_time]

    def get_all_ingredients(self):
        all_ingredients = []
        for ingredients in self.df['Ingredients']:
            all_ingredients.extend(ingredients.split(','))
        return list(set(all_ingredients))

    def check_ingredients(self, recipe_id, available_ingredients):
        recipe = self.df[self.df['Recipe_ID'] == recipe_id].iloc[0]
        required_ingredients = set(recipe['Ingredients'].split(','))
        available_ingredients = set(available_ingredients)
        missing_ingredients = required_ingredients - available_ingredients
        return list(missing_ingredients)

    def generate_shopping_list(self, recipe_ids, available_ingredients):
        shopping_list = set()
        for recipe_id in recipe_ids:
            missing = self.check_ingredients(recipe_id, available_ingredients)
            shopping_list.update(missing)
        return list(shopping_list)

    def save(self):
        self.df.to_csv(self.csv_file, index=False)


jamil
